# EX11



Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é **R$ 2,00** nas duas primeiras semanas e subirá para **R$ 2,50** nas duas últimas, quando a concorrência demandar por material no mercado. 

A demanda semanal de camisetas será de **5.000, 10.000, 30.000 e 60.000**. A capacidade máxima de produção da empresa é de **25.000 camisetas**. Na primeira e na segunda semana, a empresa poderá, em um esforço excepcional, carrear mão-de-obra em horas extras e fabricar mais **10.000 camisetas** em cada semana. Nesse caso, o custo dessas camisetas será de **R$ 2,80**. O excesso de produção pode ser estocado a um custo de **R$ 0,20** por unidade por semana.

#### Pedido 1:
Formular o modelo de PL que **minimize os custos**.



## Modelagem Matemática

### **Definição das variáveis**
Vamos definir as seguintes variáveis de decisão:

- $x_{t}$ = número de camisetas produzidas na semana $t$, onde $t = 1,2,3,4$.
- $y_{t}$ = número de camisetas produzidas com horas extras na semana $t$, onde $t = 1,2$.
- $s_{t}$ = número de camisetas estocadas no final da semana $t$, onde $t = 1,2,3,4$.

### **Função objetivo**
Queremos minimizar os custos de produção e armazenamento:

$$
\min \sum_{t=1}^{2} (2x_{t} + 2.8y_{t}) + \sum_{t=3}^{4} (2.5x_{t}) + \sum_{t=1}^{3} (0.2s_{t})
$$

### **Restrições**
1. **Capacidade de produção por semana**:
   - Semana 1 e 2 (produção normal + horas extras):
     $$
     x_{t} + y_{t} \leq 35.000, \quad \forall t = 1,2
     $$
   - Semana 3 e 4 (produção normal):
     $$
     x_{t} \leq 25.000, \quad \forall t = 3,4
     $$

2. **Balanço de estoque**:
   - Relaciona a produção, demanda e estoque de cada semana:
     $$
     x_{1} + y_{1} = 5.000 + s_{1}
     $$
     $$
     x_{2} + y_{2} + s_{1} = 10.000 + s_{2}
     $$
     $$
     x_{3} + s_{2} = 30.000 + s_{3}
     $$
     $$
     x_{4} + s_{3} = 60.000
     $$

3. **Não negatividade**:
   - Todas as variáveis devem ser não negativas:
     $$
     x_{t} \geq 0, \quad y_{t} \geq 0, \quad s_{t} \geq 0, \quad \forall t
     $$



In [40]:
import pyomo.environ as pyo

In [41]:
# Criação do modelo
model = pyo.ConcreteModel()

In [42]:

# Conjuntos
model.S = pyo.Set(initialize=[1, 2, 3, 4])
 

In [43]:
# ------------------------------
# Parâmetros
# ------------------------------
# Custos de produção por unidade
custo_producao = {1: 2, 2: 2, 3: 2.5, 4: 2.5}
model.c_prod = pyo.Param(model.S, initialize=custo_producao)

# Custos de produção em horas extras (apenas semanas 1 e 2)
custo_extra = {1: 2.8, 2: 2.8}
model.c_extra = pyo.Param(model.S, initialize=custo_extra, default=0)

# Custo de armazenamento por unidade por semana
custo_estoque = 0.2

# Demanda semanal de camisetas
demanda = {1: 5000, 2: 10000, 3: 30000, 4: 60000}
model.d = pyo.Param(model.S, initialize=demanda)

# Capacidade máxima de produção regular
capacidade_producao = 25000

# Capacidade extra de produção nas semanas 1 e 2
capacidade_extra = 10000



In [44]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# Quantidade produzida regularmente por semana
model.x = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

# Quantidade produzida com horas extras (apenas semanas 1 e 2)
model.y = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

# Quantidade estocada ao final de cada semana
model.s = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

In [45]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    custo_total_producao = sum(model.x[s] * model.c_prod[s] + model.y[s] * model.c_extra[s] for s in model.S)
    custo_total_estoque = sum(model.s[s] * custo_estoque for s in model.S if s < 4)
    return custo_total_producao + custo_total_estoque
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [46]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite de Produção Regular
def limite_producao(model, s):
    return model.x[s] <= capacidade_producao

model.CapProd = pyo.Constraint(model.S, rule=limite_producao)

# 2. Limite de Produção Extra nas semanas 1 e 2
def limite_extra(model, s):
    if s in [1, 2]:
        return model.y[s] <= capacidade_extra
    return model.y[s] == 0  # Sem produção extra nas semanas 3 e 4

model.CapExtra = pyo.Constraint(model.S, rule=limite_extra)

# 3. Balanço de Estoque
def estoque(model, s):
    if s == 1:
        return model.x[1] + model.y[1] == model.d[1] + model.s[1]
    elif s == 2:
        return model.x[2] + model.y[2] + model.s[1] == model.d[2] + model.s[2]
    elif s == 3:
        return model.x[3] + model.s[2] == model.d[3] + model.s[3]
    elif s == 4:
        return model.x[4] + model.s[3] == model.d[4]

model.BalancoEstoque = pyo.Constraint(model.S, rule=estoque)



In [47]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex11a.lp", io_options={"symbolic_solver_labels": True})

('ex11a.lp', 2271357883968)

In [ ]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)

MIP  has 12 rows; 11 cols; 20 nonzeros; 11 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e-01, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 6e+04]
Presolving model
3 rows, 8 cols, 10 nonzeros  0s
2 rows, 7 cols, 8 nonzeros  0s
1 rows, 6 cols, 6 nonzeros  0s
1 rows, 4 cols, 4 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
Objective function is integral with scale 10

Solving MIP model with:
   1 rows
   3 cols (0 binary, 3 integer, 0 implied int., 0 continuous)
   3 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves  

In [49]:
print(f"Custo total mínimo: R$ {model.objetivo():.2f}")
for t in model.S:
    print(f"\nSemana {t}:")
    print(f"Produção regular na semana {t}: {model.x[t].value} unidades")
    print(f"Produção extra na semana {t}: {model.y[t].value} unidades")
    print(f"Estoque ao final da semana {t}: {model.s[t].value} unidades")
    


Custo total mínimo: R$ 258000.00

Semana 1:
Produção regular na semana 1: 25000.0 unidades
Produção extra na semana 1: 0.0 unidades
Estoque ao final da semana 1: 20000.0 unidades

Semana 2:
Produção regular na semana 2: 25000.0 unidades
Produção extra na semana 2: 5000.0 unidades
Estoque ao final da semana 2: 40000.0 unidades

Semana 3:
Produção regular na semana 3: 25000.0 unidades
Produção extra na semana 3: 0.0 unidades
Estoque ao final da semana 3: 35000.0 unidades

Semana 4:
Produção regular na semana 4: 25000.0 unidades
Produção extra na semana 4: 0.0 unidades
Estoque ao final da semana 4: None unidades


#### Pedido 2:
Após o planejamento anterior, a direção da empresa verificou que a demanda iria variar substancialmente dentro dos quatro modelos de camiseta que representavam os quatro times disputando as finais. Apesar da demanda total ser exatamente aquela anteriormente levantada, o valor das camisetas iria variar em conformidade com o time e sua posição no campeonato. Nas duas primeiras semanas, todos os times estariam em pé de igualdade até que fossem decididos os dois finalistas. A partir daí, as camisetas dos times eliminados **cairiam** em valor e em demanda no mercado, e as dos times finalistas **subiriam** conforme a tabela a seguir:

| Semana | 1       | 1       |2       |2       | 3       |3       | 4       |4       |
|--------|--------|--------|--------|--------|--------|--------|--------|--------|
|        | Demanda | Valor  | Demanda | Valor  | Demanda | Valor  | Demanda | Valor  |
| **Time A** | 1250  | 5,00  | 2500  | 6,00  | 5000  | 3,00  | -  | -  |
| **Time B** | 1250  | 5,00  | 2500  | 6,00  | 5000  | 3,00  | -  | - |
| **Time Final C** | 1250  | 5,00  | 2500  | 6,00  | 14500 | 8,00  | 30000 | 9,00  |
| **Time Final D** | 1250  | 5,00  | 2500  | 6,00  | 14500 | 8,00  | 30000 | 9,00  |

Sabendo-se que existe um completo equilíbrio entre quatro finalistas, formular o modelo qye maximize os lucros da empresa produtora de camisetas

## Modelagem Matemática para Maximização do Lucro

### **Definição das variáveis**
Vamos definir as seguintes variáveis de decisão:

- $x_{t,i}$ = número de camisetas produzidas do time $i$ na semana $t$, onde $t = 1,2,3,4$ e $i \in \{A, B, C, D\}$.
- $s_{t,i}$ = número de camisetas estocadas do time $i$ no final da semana $t$.
- $y_{t}$ = número de camisetas produzidas com horas extras na semana $t$ (apenas semanas 1 e 2).

### **Parâmetros**
- $v_{t,i}$ = preço de venda da camiseta do time $i$ na semana $t$.
- $d_{t,i}$ = demanda da camiseta do time $i$ na semana $t$.
- Custo de produção:
  - Nas semanas 1 e 2: **R$ 2,00**
  - Nas semanas 3 e 4: **R$ 2,50**
  - Produção em horas extras (semanas 1 e 2): **R$ 2,80**
- Custo de estocagem: **R$ 0,20** por unidade por semana.
- Capacidade máxima de produção regular: **25.000** camisetas.
- Produção extra disponível nas semanas 1 e 2: **10.000** camisetas.

### **Função Objetivo**
Queremos **maximizar o lucro**, que é a diferença entre receita e custo:

$$
\max \sum_{t=1}^{4} \sum_{i \in \{A, B, C, D\}} v_{t,i} \cdot x_{t,i} - \sum_{t=1}^{2} (2x_{t} + 2.8y_{t}) - \sum_{t=3}^{4} (2.5x_{t}) - \sum_{t=1}^{3} (0.2s_{t})
$$

### **Restrições**
1. **Capacidade de produção por semana**:
   - Semana 1 e 2 (produção normal + horas extras):
     $$
     \sum_{i \in \{A, B, C, D\}} x_{t,i} + y_{t} \leq 35.000, \quad \forall t = 1,2
     $$
   - Semana 3 e 4 (produção normal):
     $$
     \sum_{i \in \{A, B, C, D\}} x_{t,i} \leq 25.000, \quad \forall t = 3,4
     $$

2. **Balanço de estoque**:
   - Relaciona a produção, demanda e estoque de cada semana para cada time:
     $$
     x_{t,i} + s_{t-1,i} = d_{t,i} + s_{t,i}, \quad \forall t, i
     $$

3. **Não negatividade**:
   - Todas as variáveis devem ser não negativas:
     $$
     x_{t,i} \geq 0, \quad s_{t,i} \geq 0, \quad y_{t} \geq 0, \quad \forall t, i
     $$



In [50]:
# Criação do modelo
model = pyo.ConcreteModel()

In [51]:

# Conjuntos
model.S = pyo.Set(initialize=[1, 2, 3, 4])  # Semanas
model.T = pyo.Set(initialize=["A", "B", "C", "D"])  # Times


In [52]:
# ------------------------------
# Parâmetros
# ------------------------------
# Preço de venda das camisetas por semana
preco_venda = {
    (1, "A"): 5, (2, "A"): 6, (3, "A"): 3, (4, "A"): 0,
    (1, "B"): 5, (2, "B"): 6, (3, "B"): 3, (4, "B"): 0,
    (1, "C"): 5, (2, "C"): 6, (3, "C"): 8, (4, "C"): 9,
    (1, "D"): 5, (2, "D"): 6, (3, "D"): 8, (4, "D"): 9
}
model.v = pyo.Param(model.S, model.T, initialize=preco_venda)

# Demanda por semana e time
demanda = {
    (1, "A"): 1250, (2, "A"): 2500, (3, "A"): 5000, (4, "A"): 0,
    (1, "B"): 1250, (2, "B"): 2500, (3, "B"): 5000, (4, "B"): 0,
    (1, "C"): 1250, (2, "C"): 2500, (3, "C"): 14500, (4, "C"): 30000,
    (1, "D"): 1250, (2, "D"): 2500, (3, "D"): 14500, (4, "D"): 30000
}
model.d = pyo.Param(model.S, model.T, initialize=demanda)

# Custos de produção
custo_producao = {1: 2, 2: 2, 3: 2.5, 4: 2.5}
model.c_prod = pyo.Param(model.S, initialize=custo_producao)

# Custos de produção em horas extras (apenas semanas 1 e 2)
custo_extra = {1: 2.8, 2: 2.8}
model.c_extra = pyo.Param(model.S, initialize=custo_extra, default=0)

# Custo de armazenamento por unidade por semana
custo_estoque = 0.2

# Capacidade máxima de produção regular
capacidade_producao = 25000

# Produção extra disponível nas semanas 1 e 2
capacidade_extra = 10000



In [54]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
# Quantidade produzida por semana e time
model.x = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)

# Quantidade estocada ao final de cada semana por time
model.s = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)

# Quantidade produzida com horas extras (apenas semanas 1 e 2)
model.y = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

In [55]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    
    receita_total = sum(model.v[s, t] * model.x[s, t] for s in model.S for t in model.T)

    
    custo_total_producao = sum(model.x[s, t] * model.c_prod[s] for s in model.S for t in model.T)
    custo_total_extra = sum(model.y[s] * model.c_extra[s] for s in model.S if s in [1, 2])
    custo_total_estoque = sum(model.s[s, t] * custo_estoque for s in model.S for t in model.T if s < 4)

    return receita_total - (custo_total_producao + custo_total_extra + custo_total_estoque)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [56]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Limite de produção por semana
def limite_producao(model, s):
    return sum(model.x[s, t] for t in model.T) + (model.y[s] if s in [1, 2] else 0) <= (capacidade_producao + (capacidade_extra if s in [1, 2] else 0))

model.CapProd = pyo.Constraint(model.S, rule=limite_producao)

# 2. Balanço de estoque por time
def estoque(model, s, t):
    if s == 1:
        return model.x[1, t] == model.d[1, t] + model.s[1, t]
    elif s == 2:
        return model.x[2, t] + model.s[1, t] == model.d[2, t] + model.s[2, t]
    elif s == 3:
        return model.x[3, t] + model.s[2, t] == model.d[3, t] + model.s[3, t]
    elif s == 4:
        return model.x[4, t] + model.s[3, t] == model.d[4, t]

model.BalancoEstoque = pyo.Constraint(model.S, model.T, rule=estoque)




In [57]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex11b.lp", io_options={"symbolic_solver_labels": True})


('ex11b.lp', 2271358555136)

In [58]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 20 rows; 30 cols; 58 nonzeros; 30 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e-01, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 4e+04]
Presolving model
12 rows, 18 cols, 36 nonzeros  0s
10 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 10

Solving MIP model with:
   10 rows
   16 cols (0 binary, 16 integer, 0 implied int., 0 continuous)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0    

In [61]:




# Exibindo os valores das variáveis
print("\nProdução semanal por time:")
for s in model.S:
    for t in model.T:
        print(f"Semana {s}, Time {t}: {model.x[s, t].value} camisetas")

print("\nEstoque final por time:")
for s in model.S:
    for t in model.T:
        print(f"Semana {s}, Time {t}: {model.s[s, t].value} camisetas")

print("\nProdução extra:")
for s in model.S:
    print(f"Semana {s}: {model.y[s].value} camisetas (extra)")




Produção semanal por time:
Semana 1, Time A: 1250.0 camisetas
Semana 1, Time B: 8750.0 camisetas
Semana 1, Time C: 1250.0 camisetas
Semana 1, Time D: 17750.0 camisetas
Semana 2, Time A: 7500.0 camisetas
Semana 2, Time B: -0.0 camisetas
Semana 2, Time C: 27500.0 camisetas
Semana 2, Time D: 0.0 camisetas
Semana 3, Time A: 0.0 camisetas
Semana 3, Time B: 0.0 camisetas
Semana 3, Time C: 0.0 camisetas
Semana 3, Time D: 25000.0 camisetas
Semana 4, Time A: 0.0 camisetas
Semana 4, Time B: 0.0 camisetas
Semana 4, Time C: 19500.0 camisetas
Semana 4, Time D: 5500.0 camisetas

Estoque final por time:
Semana 1, Time A: 0.0 camisetas
Semana 1, Time B: 7500.0 camisetas
Semana 1, Time C: 0.0 camisetas
Semana 1, Time D: 16500.0 camisetas
Semana 2, Time A: 5000.0 camisetas
Semana 2, Time B: 5000.0 camisetas
Semana 2, Time C: 25000.0 camisetas
Semana 2, Time D: 14000.0 camisetas
Semana 3, Time A: 0.0 camisetas
Semana 3, Time B: 0.0 camisetas
Semana 3, Time C: 10500.0 camisetas
Semana 3, Time D: 24500.0 

In [63]:
print("\nResultados por semana:")
for s in model.S:
    print(f"\n Semana {s}")
    
    print("Produção por time:")
    for t in model.T:
        print(f"  - Time {t}: {model.x[s, t].value} camisetas")
    print("Estoque final por time:")
    for t in model.T:
        print(f"  - Time {t}: {model.s[s, t].value} camisetas")
    print(f"Produção extra na semana {s}: {model.y[s].value} camisetas")



Resultados por semana:

 Semana 1
Produção por time:
  - Time A: 1250.0 camisetas
  - Time B: 8750.0 camisetas
  - Time C: 1250.0 camisetas
  - Time D: 17750.0 camisetas
Estoque final por time:
  - Time A: 0.0 camisetas
  - Time B: 7500.0 camisetas
  - Time C: 0.0 camisetas
  - Time D: 16500.0 camisetas
Produção extra na semana 1: 0.0 camisetas

 Semana 2
Produção por time:
  - Time A: 7500.0 camisetas
  - Time B: -0.0 camisetas
  - Time C: 27500.0 camisetas
  - Time D: 0.0 camisetas
Estoque final por time:
  - Time A: 5000.0 camisetas
  - Time B: 5000.0 camisetas
  - Time C: 25000.0 camisetas
  - Time D: 14000.0 camisetas
Produção extra na semana 2: 0.0 camisetas

 Semana 3
Produção por time:
  - Time A: 0.0 camisetas
  - Time B: 0.0 camisetas
  - Time C: 0.0 camisetas
  - Time D: 25000.0 camisetas
Estoque final por time:
  - Time A: 0.0 camisetas
  - Time B: 0.0 camisetas
  - Time C: 10500.0 camisetas
  - Time D: 24500.0 camisetas
Produção extra na semana 3: None camisetas

 Semana 